# Computational Models for Complex Systems Project

## Implementation of a SIRVSD model with different age goups

### The model
$$
\begin{equation}
    \begin{cases}
    \frac{\textit{dS}_j}{\textit{dt}} = \phi \textit{R}_j - \eta_j (t) \textit{S}_j + \rho \textit{V}_j - \textit{S}_j \sum_{k=1}^M \beta_{j,k} \textit{I}_k \\[4pt]
    \frac{\textit{dI}_j}{\textit{dt}} = - \gamma \textit{I}_j - \mu_j \textit{I}_j + \textit{S}_j \sum_{k=1}^M \beta_{j,k} \textit{I}_k \\[4pt]
    \frac{\textit{dR}_j}{\textit{dt}} = \gamma \textit{I}_j - \phi \textit{R}_j & \hspace{10pt} \forall j \in \left(0, M\right] \\[4pt]
    \frac{\textit{dV}_j}{\textit{dt}} = \eta_j (t) \textit{S}_j - \rho \textit{V}_j \\[4pt]
    \frac{\textit{dD}_j}{\textit{dt}} = \mu_j \textit{I}_j \\[4pt]
    \end{cases}
\end{equation}
$$

with $M$ = 4 age groups: Children, Teenagers, Adults and Senior Citizens.

### Definition of parameters

In our system, we have a lot of different coefficients:
- $\phi$ is the $\textit{transfer rate}$ for loss of immunity from Recovered.
- $\rho$ is the $\textit{transfer rate}$ for loss of immunity from Vaccinated.
- $\beta_{j, k}$ is the $\textit{infection rate}$, computed as the average number of contacts per person per time, multiplied by the probability of disease transmission in a contact between a susceptible subject of group $j$ and an infected subject of group $k$. <br>
We define also the entire $\textit{infection rate matrix}$ $\beta$ (or $\textit{contact matrix}$): 
$$\beta = \begin{bmatrix}
        \beta_{1, 1} & \cdots & \beta_{1, M}\\
        \vdots & \ddots & \vdots \\
        \beta_{M, 1} & \cdots & \beta_{M, M}
\end{bmatrix}
$$
- $\gamma$ is the $\textit{recovery rate}$ of each infected subject.
- $\mu_j$ is the $\textit{mortality rate}$, different for each age group.

The last coefficient is $\eta_j(t)$, a time-dependent $\textit{vaccination rate}$, defines as follows:
$$
    \begin{equation}
        \eta_j (t) = 
        \begin{cases}
            0 &\quad\text{if } t < t_{\textit{vacc}_j}\\
            \eta_j &\quad\text{otherwise}
        \end{cases}
    \end{equation}
$$

where $t_{\textit{vacc}_j}$ defines the starting day of the vaccination period.

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [2]:
# TODO try to implement also a solve_ivp solver
def sir_solver(t,beta_matrix,gamma,mu_group,phi, rho,eta_group,x0,start_vaccination):

    def assign_vaccination_rate(t, eta, start_vaccination):
        """
        Auxiliary function to assign time-dependent vaccination rate eta
        """
        if t < start_vaccination or start_vaccination == -1: # -1 means no vaccination for a specific age group
            return 0
        else:
            return eta
    
    def sir(x,t,beta_matrix,gamma,mu_group,phi,rho,eta_group,start_vaccination,assign_vaccination_rate):
        n_groups = len(start_vaccination) # or any other "_group" parameter
        derivatives_matrix = np.zeros((5,4)) # save all derivatives in a 2-D array
        n_infectious = [x[j+n_groups] for j in range(0,n_groups)] # list of the infectious measured for each group
        for j in range(0,n_groups):
            s = x[j] # Susceptible
            # i = n_infectious[j]
            i = x[j+n_groups] # Infectious
            r = x[j+n_groups*2] # Recovered
            v = x[j+n_groups*3] # Vaccinated
            d = x[j+n_groups*4] # Deceased
            eta = assign_vaccination_rate(t,eta_group[j],start_vaccination[j]) # time-dependent parameter
            derivatives_matrix[0][j] = phi*r - eta*s + rho*v - s*np.dot(beta_matrix[j],n_infectious) # dsdt
            derivatives_matrix[1][j] = s*np.dot(beta_matrix[j],n_infectious) - gamma*i - mu_group[j]*i # didt
            derivatives_matrix[2][j] = gamma*i - phi*r # drdt
            derivatives_matrix[3][j] = eta*s - rho*v # dvdt
            derivatives_matrix[4][j] = mu_group[j]*i # dddt
        return derivatives_matrix.reshape(-1) # return all measurements with a 1-D array

    y = odeint(sir,x0,t,args=(beta_matrix,gamma,mu_group,phi,rho,eta_group,start_vaccination,assign_vaccination_rate,)) 
    return y 

### Definition of constants

We considered these groups:
- Children (0-9 years)
- Teenagers (10-19 years)
- Adults (20-69 years)
- Senior Citizens (70+ years)

In the following initial conditions and parameters, we define an indexing like this:

    S_0_GROUP = [x, y, w, y]
        - x is the intitial number of susceptible in "Children" group
        - y is the intitial number of susceptible in "Teenagers" group
        - w is the intitial number of susceptible in "Adults" group
        - z is the intitial number of susceptible in "Senior Citizens" group

We can change the value of the costants, according to the experiments we want to run and the graphs we want to analyze.

In [3]:
group_dict = {
    "children": 0,
    "teenagers": 1,
    "adults": 2,
    "senior": 3
}
START = 0 # day of start of the observation period
END = 365 # day of end of the observation period
START_VACCINATION_GROUP = [-1, -1, -1, -1] # day of start of the vaccination period (-1 -> no vaccination)

# initial conditions
S_0_GROUP = [0.99, 0.99, 0.99, 0.99] # Susceptible
I_0_GROUP = [0.01, 0.01, 0.01, 0.01] # Infectious
R_0_GROUP = [0, 0, 0, 0] # Recovered
V_0_GROUP = [0, 0, 0, 0] # Vaccinated
D_0_GROUP = [0, 0, 0, 0] # Deceased

# model parameters
beta_matrix = np.array([[0.6,0,0,0],[0,0.6,0,0],[0,0,0.6,0],[0,0,0,0.6]]) # infection coefficient for each group
gamma = 1/15 # recovery coefficient (same for all group)
mu_group = [0.00009, 0.00005, 0.00688, 0.15987] # mortality coefficient for each group (case fataly rate ISS report January 2021)
phi = 1/180 # transfer rate for loss of immunity from recovered (six months of immunity and same for all group)
rho = 1/270 # transfer rate for loss of immunity from vaccinated (nine months of immunity and same for all group)
eta_group = [0.003, 0.003, 0.003, 0.01] # vaccination rate for each group

t = np.linspace(START,END,END-START+1) # setting the simulation time and the number of points

### Function call

In [4]:
results_dict = {}
x_0 = [*S_0_GROUP, *I_0_GROUP, *R_0_GROUP, *V_0_GROUP, *D_0_GROUP] # unpacking list operator
y = sir_solver(t, beta_matrix, gamma, mu_group, phi, rho, eta_group, x_0, START_VACCINATION_GROUP)
_, n_total_column = y.shape
n_groups = len(group_dict) # number of age groups
n_compartments = int(n_total_column/n_groups) # number of compartments of the model
for group_name, group_id in group_dict.items():
    # select the right columns (the compartments) for each age group 
    results_dict[group_name] = y[:,[group_id+n_groups*j for j in range(0,n_compartments)]]